In [ ]:
# скачал Access базу и перевел её в csv с помощью сайта https://www.rebasedata.com/convert-mdb-to-csv-online
# pyodbc не работал из-за отсутствия драйвера
# from mdb_parser import MDBParser, MDBTable - надо устанавливать  mdb_parser и читает вроде по одной таблице, 
# а там их много в базе...?

In [1]:
import sqlite3
import pandas as pd

In [2]:
# connect
conn = sqlite3.connect('db')

In [27]:
# заливка таблицы  в  'db' базу
path = '/Users/maksa/Downloads/sql_ex_base/Printer.csv'
dbase = pd.read_csv(path)
name_base = path.rsplit('/',1)[-1].rsplit('.',1)[0]
dbase.to_sql(name_base, conn, index=False, if_exists='replace')

In [ ]:
# Product(maker, model, type)
# PC(code, model, speed, ram, hd, cd, price)
# Laptop(code, model, speed, ram, hd, price, screen)
# Printer(code, model, color, type, price)

In [3]:
# функция select, для короткой записи запроса
def select(sql):
    return pd.read_sql(sql, conn)

In [ ]:
Найдите производителей, которые производили бы как ПК
со скоростью не менее 750 МГц, так и ПК-блокноты со скоростью не менее 750 МГц.
Вывести: Maker

In [26]:
# INTERSECT [ALL] (пересечение) и EXCEPT [ALL] (разность), которые работают аналогично предложению UNION

sql = '''select maker
from product as pr
left join pc on pr.model = pc.model
where  pc.speed >= 750
INTERSECT
select maker
from product as pr
left join laptop as lp on pr.model = lp.model
where lp.speed >= 750
'''
select(sql)

,maker
0,A
1,B


In [ ]:
Перечислите номера моделей любых типов, имеющих самую высокую цену по всей имеющейся в базе данных продукции.

In [52]:
sql = '''
with 
max_pc as (select model, max(price) from pc),
max_lp as (select model, max(price) from laptop),
max_tr as (select model, max(price) from printer)
select model
from max_pc
union
select model
from max_lp
union
select model
from max_tr
'''
select(sql)

,model
0,1233
1,1276
2,1750


In [82]:
# # верный для postgresql
sql = '''
with total as
(select pr.model, price
from product as pr
left join pc on pr.model = pc.model
group by pr.model, price
union
select pr.model, price
from product as pr
left join laptop as lp on pr.model = lp.model
group by pr.model, price
union
select pr.model, price
from product as pr
left join printer as t on pr.model = t.model
group by pr.model, price)

select model from total
where price = (select max(price) from total)
'''
select(sql)

,model
0,1750


In [ ]:
Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price

In [108]:
# правильные данные на основной базе и на проверочной базе.

sql = '''
with min_price as
(select model, price 
from printer
where color = 'y' and price = (select min(price) from printer where color = 'y')
group by model, price
)

select distinct maker, price
from product as pr
inner join min_price as mp on pr.model = mp.model'''
select(sql)

,maker,price
0,D,270


In [107]:
# правильные данные на основной базе, но не прошел тест на проверочной базе.
# Неверное число записей (больше на 2)
sql = '''SELECT distinct c.maker, a.priceA price
FROM (SELECT MIN(price) priceA 
      FROM Printer 
      WHERE color ='y'
      ) a INNER JOIN 
      (select * from Printer WHERE color ='y') b ON a.priceA = b.price
      INNER JOIN 
      Product c ON b.model = c.model'''
select(sql)

,maker,price
0,D,270


In [ ]:
Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.

In [124]:
#  ???
sql = '''
with mm as
(select maker, pc.model
from pc 
left join product pr on pc.model = pr.model)

select maker, count(distinct model)
from mm
group by maker
-- having count(distinct model) >= 3
'''
select(sql)

,maker,count(distinct model)
0,A,2
1,B,1
2,E,1


In [148]:
sql = '''
SELECT maker, COUNT(model)
FROM Product
GROUP BY maker, type
HAVING COUNT(model) >= 3 AND type ='PC'
'''
select(sql)

,maker,COUNT(model)
0,E,3


In [ ]:
Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и 
с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker

In [156]:
sql = '''
select *
from product 
where type = 'Printer'
--inner join product pr on printer.model = pr.model
'''
select(sql)

,maker,model,type
0,A,1276,Printer
1,D,1288,Printer
2,A,1401,Printer
3,A,1408,Printer
4,D,1433,Printer
5,E,1434,Printer


In [150]:
sql = '''
select *
from pc 
inner join product pr on pc.model = pr.model
-- group by speed
where ram = (select min(ram) from pc) 

'''
select(sql)

,code,model,speed,ram,hd,cd,price,maker,model,type
0,7,1232,500,32,10,12x,400,A,1232,PC
1,9,1232,450,32,10,24x,350,A,1232,PC
2,10,1260,500,32,10,12x,350,E,1260,PC


In [158]:
sql = '''
with ram as
(select speed, maker
from pc 
left join product pr on pc.model = pr.model
-- group by speed
where ram = (select min(ram) from pc)) 

select distinct maker
from ram
where speed = (select max(speed) from ram)
INTERSECT
select distinct maker
from product 
where type = 'Printer'

'''
select(sql)

,maker
0,A
1,E


In [ ]:
Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена.

In [186]:
sql = '''
select *
from product 
where type in ('PC', 'Laptop') and maker = 'A'
--inner join product pr on printer.model = pr.model
'''
select(sql)

,maker,model,type
0,A,1232,PC
1,A,1233,PC
2,A,1298,Laptop
3,A,1752,Laptop


In [180]:
sql = '''
with Aa as 
(select *
from product 
where type in ('PC', 'Laptop') and maker = 'A'),

a_price as (select maker, Aa.model, price
from Aa 
inner join pc on Aa.model = pc.model
union all                                -- 'all'  выводит всё с дубликатами (это и нужно)
select maker, Aa.model, price
from Aa 
inner join laptop lp on Aa.model = lp.model)

select avg(price) as AVG
from a_price
'''
select(sql)

,AVG
0,773.0


In [183]:
# этот тоже правильно...
sql ='''select avg(price) AVG_price from product p,
(select model, price from Laptop union all select model, price from pc) t
where p.model = t.model and p.maker = 'A'
'''
select(sql)

,AVG_price
0,734.545455


In [ ]:
Найдите средний размер диска ПК каждого из тех производителей, которые выпускают и принтеры. Вывести: maker, средний размер HD.

In [193]:
sql = '''select distinct maker, avg(hd)
from Product pr, PC
where pr.model = pc.model and pr.maker in (select maker
                                            from Product
                                            where type = 'printer')
group by maker'''
select(sql)

,maker,avg(hd)


In [ ]:
Для каждого производителя, у которого присутствуют модели хотя бы в одной из таблиц PC, Laptop или Printer,
определить максимальную цену на его продукцию.
Вывод: имя производителя, если среди цен на продукцию данного производителя присутствует NULL, то выводить для этого производителя NULL,
иначе максимальную цену.


In [205]:
sql = '''select maker,  pc.price -- max(lp.price)
from Product pr, PC --, Laptop lp --, Printer
where pr.model = pc.model 

--pr.maker in (select maker
--                                            from Product
--                                            where type = 'printer')
--group by maker'''
select(sql)

,maker,price
0,B,850
1,B,850
2,B,850
3,A,350
4,A,350
5,A,400
6,A,600
7,A,600
8,A,950
9,A,980


In [257]:
# Здесь case верный, в остальных нет (во всяком случае для postgresql)
sql = '''
with all_price as
(select maker, price
from Product pr, PC
where pr.model = pc.model
group by maker, pc.price
union all
select maker, price
from Product pr, Laptop lp
where pr.model = lp.model
group by maker, price
union all
select maker, price
from Product pr, Printer pt
where pr.model = pt.model
group by maker, price)

select maker, case  when count(price) = count(*) then max(price) end price 
from all_price
group by maker 
 '''


select(sql)

,maker,price
0,A,1150
1,B,1200
2,C,970
3,D,400
4,E,350


In [258]:
sql = '''
with all_price as
(select model, price from PC
union all
select model, price from Laptop
union all
select model, price from Printer
)

select distinct maker, case  when count(price) = count(*) then max(price) end price 
from Product pr, all_price ap
where pr.model = ap.model
group by pr.maker
--group by maker 
 '''
select(sql)

,maker,price
0,A,1150
1,B,1200
2,C,970
3,D,400
4,E,350


In [256]:
sql = '''
with all_price as
(select model, price from PC
union all
select model, price from Laptop
union all
select model, price from Printer
)

select distinct maker, case WHEN price IS NULL 
                            THEN 0
                            ELSE max(price)
                            END price 
from Product pr
join all_price ap on pr.model=ap.model
group by pr.maker
 '''
select(sql)

,maker,price
0,A,1150
1,B,1200
2,C,970
3,D,400
4,E,350
